In [1]:
#import the dataset and strip off unwanted columns 
import pandas as pd

df = pd.read_csv('US_Accidents_March23.csv', dtype={"fips": str})
df = df[['Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Street', 'City', 'County', 'State', 'Zipcode', 'Temperature(F)','Wind_Chill(F)', 'Humidity(%)','Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Crossing', 'Junction', 'Stop', 'Traffic_Signal', 'Sunrise_Sunset']]

In [ ]:
df[['Zipcode', 'Mailing_Zip']] = df['Zipcode'].str.split('-', expand=True)


In [ ]:
import numpy as np
county_entry_counts = df.groupby('Zipcode').size().reset_index(name='entry_count')

county_entry_counts['entry_count_scaled'] = np.log10(county_entry_counts['entry_count'])

print(county_entry_counts)


In [27]:
#import the geojson for US counties 
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
print(json.dumps(counties, indent=2))

In [45]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'firefox'

fig = px.choropleth_mapbox(
    county_entry_counts,
    geojson=counties,
    locations= county_entry_counts['Zipcode'], 
    color='entry_count_scaled', 
    hover_name='Zipcode', 
    mapbox_style="carto-positron",
    center=dict(lat=37.0902, lon=-95.7129),
    zoom=3,
    opacity=0.7,
    title='Density Car Crash Map of the Continental US'
)

fig.update_layout(
    coloraxis_colorbar=dict(
        title='Density'
        # Adjust tick labels as needed
    )
)

fig.show()

In [28]:
import numpy as np
state_entry_counts = df.groupby('State').size().reset_index(name='entry_count')

#county_entry_counts['entry_count_scaled'] = np.log10(county_entry_counts['entry_count'])
state_entry_counts['State'] = (state_entry_counts.index + 1).map(lambda x: '{:02d}'.format(x))

print(state_entry_counts)

   State  entry_count
0     01       101044
1     02        22780
2     03       170609
3     04      1741433
4     05        90885
5     06        71005
6     07        18630
7     08        14097
8     09       880192
9     10       169234
10    11        26307
11    12        11376
12    13       168958
13    14        67224
14    15        20992
15    16        32254
16    17       149701
17    18        61996
18    19       140417
19    20         2698
20    21       162191
21    22       192084
22    23        77323
23    24        15181
24    25        28496
25    26       338199
26    27         3487
27    28        28870
28    29        10213
29    30       140719
30    31        10325
31    32        21665
32    33       347960
33    34       118115
34    35        83647
35    36       179660
36    37       296620
37    38        16971
38    39       382557
39    40          289
40    41       167388
41    42       582837
42    43        97079
43    44       303301
44    45  

In [31]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'firefox'

fig = px.choropleth_mapbox(
    state_entry_counts,
    geojson=counties,
    locations= state_entry_counts['State'],  # Replace 'FIPS' with the actual column name in your DataFrame
    color='entry_count',  # Replace 'Density' with the column you want to visualize
    hover_name='State',  # Replace 'County' with the column containing county names
    mapbox_style="carto-positron",
    center=dict(lat=37.0902, lon=-95.7129),
    zoom=3,
    opacity=0.7,
    title='Density Car Crash Map of the Continental US'
)

fig.update_layout(
    coloraxis_colorbar=dict(
        title='Density'
        # Adjust tick labels as needed
    )
)

fig.show()

In [9]:
from dash import Dash, html, dash_table, dcc
import plotly.express as px
import pandas as pd